In [66]:
# Data manipulation
import numpy as np
import pandas as pd

# Data Visualiation
import matplotlib.pyplot as plt
import seaborn as sns

# System
import os

# Performance metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

from preprocessing import create_and_preprocess_X_y, create_train_test_set
from model import create_y_pred_baseline, calculate_metric, baseline_metric_score


# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential

from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import LSTM, Flatten, Dense

In [67]:
# Load and preprocess the data using create_and_preprocess_X_y()
X, y = create_and_preprocess_X_y()

In [68]:
X_train, X_test, y_train, y_test = create_train_test_set(X, y, train_size=0.8)

In [69]:
y_train_pred_baseline, y_test_pred_baseline = create_y_pred_baseline(0.8)

In [70]:
y_test_pred_baseline.shape

(18, 1, 106)

In [71]:
y_test.shape

(18, 1, 106)

In [72]:
np.unique(y_test_pred_baseline, return_counts=True)

(array([0., 1.]), array([1862,   46]))

In [73]:
np.unique(y_test, return_counts=True)

(array([0., 1.]), array([1853,   55]))

In [74]:
score = calculate_metric(y_test_pred_baseline, y_test)

In [75]:
score

0.04979035638915094

In [76]:
# Define the LSTM model
model = Sequential([
    layers.Masking(mask_value=-1, input_shape=(X_train.shape[1], X_train.shape[2])),
    layers.LSTM(units=64, activation ='tanh', return_sequences=True),
    layers.LSTM(units=64, activation ='tanh', return_sequences=True),
    layers.LSTM(units=64, activation ='tanh', return_sequences=False),
    layers.RepeatVector(1),
    layers.TimeDistributed(layers.Dense(units=106, activation = "softmax"))
])

In [77]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_3 (Masking)         (None, 200, 106)          0         
                                                                 
 lstm_8 (LSTM)               (None, 200, 64)           43776     
                                                                 
 lstm_9 (LSTM)               (None, 200, 64)           33024     
                                                                 
 lstm_10 (LSTM)              (None, 64)                33024     
                                                                 
 repeat_vector_3 (RepeatVec  (None, 1, 64)             0         
 tor)                                                            
                                                                 
 time_distributed_3 (TimeDi  (None, 1, 106)            6890      
 stributed)                                           

In [78]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [79]:
es = EarlyStopping(patience=20, restore_best_weights=True)

In [80]:
# Train the model
model.fit(X_train, y_train, epochs=500, batch_size=32,validation_split=0.2,
          callbacks=[es]) 

Epoch 1/500


2/2 [==============================] - 19s 6s/step - loss: 11.9771 - accuracy: 0.0179 - val_loss: 11.6061 - val_accuracy: 0.0000e+00
Epoch 2/500
2/2 [==============================] - 2s 1s/step - loss: 11.8105 - accuracy: 0.0179 - val_loss: 11.3754 - val_accuracy: 0.0000e+00
Epoch 3/500
2/2 [==============================] - 2s 1s/step - loss: 11.3593 - accuracy: 0.0179 - val_loss: 11.0406 - val_accuracy: 0.0000e+00
Epoch 4/500
2/2 [==============================] - 2s 1s/step - loss: 10.9102 - accuracy: 0.0179 - val_loss: 10.9205 - val_accuracy: 0.0000e+00
Epoch 5/500
2/2 [==============================] - 2s 1s/step - loss: 10.6149 - accuracy: 0.0179 - val_loss: 10.8155 - val_accuracy: 0.0000e+00
Epoch 6/500
2/2 [==============================] - 1s 787ms/step - loss: 10.3736 - accuracy: 0.0179 - val_loss: 10.7575 - val_accuracy: 0.0000e+00
Epoch 7/500
2/2 [==============================] - 1s 875ms/step - loss: 10.2283 - accuracy: 0.0536 - val_loss: 10.7510 - val_accuracy: 0.0000e+

In [81]:
# Make predictions test 
y_test_pred = model.predict(X_test)

1/1 [==============================] - 5s 5s/step


In [82]:
model_score = calculate_metric(y_test_pred, y_test)

In [83]:
model_score

0.9990453258840095

In [84]:
y_test_pred

array([[[0.00280891, 0.00109556, 0.00374138, ..., 0.00190266,
         0.00173814, 0.00301739]],

       [[0.00280179, 0.00109674, 0.00378133, ..., 0.00191105,
         0.001739  , 0.00303386]],

       [[0.0027585 , 0.00107522, 0.00371873, ..., 0.00188198,
         0.00169182, 0.00291621]],

       ...,

       [[0.00281388, 0.00110442, 0.00367768, ..., 0.00190141,
         0.00170743, 0.00294573]],

       [[0.00278159, 0.00108634, 0.00372981, ..., 0.0018924 ,
         0.00170754, 0.0029482 ]],

       [[0.00281189, 0.00110084, 0.00384602, ..., 0.00191109,
         0.00171244, 0.00300724]]], dtype=float32)

In [85]:
values, counts = np.unique(X, return_counts=True)
print(values, counts)

[0. 1.] [1819863   45737]


In [86]:
counts[1]/counts[0]

0.025132111593015517

In [87]:
X.shape

(88, 200, 106)

In [88]:
y_test_pred.shape

(18, 1, 106)

In [89]:
values_ypred, counts_ypred = np.unique(y_test_pred, return_counts=True)